In [1]:
#THIS TRAINS A HYDROLOGICAL MODEL LOCALLY, USING ERA5 AND GLOFAS DATA TO PREDICT GLOFAS DATA
import sys
sys.path.append("../../")

#HYPERPARAMETERS
days_intake_length = 180
forecast_day = 30


In [2]:
#LIBRARY IMPORTS
from functions.floodmodels import shift_input, shift_test_inputs, reshape_scalar_predictand
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from dask.distributed import Client, LocalCluster


#Connecting to a cluster to be able to run the code locally/on the cloud
client = Client()
#Use this line of code if you want to run the code on the cluster
# client = Client("tcp://169.45.50.121:8786")



In [3]:
print(client.scheduler_info()['services'])


{'dashboard': 8787}


In [4]:
ds = xr.open_dataset('../../data/features_xy.nc')


In [7]:
ds

<xarray.Dataset>
Dimensions:     (time: 13514)
Coordinates:
  * time        (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2017-12-31
Data variables:
    cp          (time) float32 0.0027969202 0.0010736011 ... 6.3911815e-05
    lsp         (time) float32 0.0030839704 0.01112505 ... 0.0028783886
    swvl1       (time) float32 0.36975124 0.36971572 ... 0.37917453 0.3986332
    ro          (time) float32 4.0711366e-05 4.0798546e-05 ... 6.1603096e-05
    sd          (time) float32 0.050891068 0.06464155 0.061856583 ... nan nan
    sd_diff     (time) float32 nan 0.013750497 -0.0027849725 ... nan nan nan
    lsp-4-11    (time) float32 nan nan nan ... 0.0021004332 0.0024646148
    lsp-12-25   (time) float32 nan nan nan ... 0.003055691 0.00295202
    lsp-26-55   (time) float32 nan nan nan ... 0.003511974 0.0031857586
    lsp-56-180  (time) float32 nan nan nan ... 0.0020519472 0.0021398212
    lsp-1       (time) float32 nan 0.0030839704 ... 0.0012526653 0.006716862
    lsp-2       (time) float32 nan nan 0.0030839704 ... 0.010335923 0.0012526653
    lsp-3       (time) float32 nan nan nan ... 0.0041955533 0.010335923
    cp-1        (time) float32 nan 0.0027969202 ... 0.00060784887 0.0015134838
    cp-2        (time) float32 nan nan ... 0.00068914227 0.00060784887
    cp-3        (time) float32 nan nan nan ... 0.00018165565 0.00068914227
    dis         (time) float64 ...
    dis_diff    (time) float64 ...

In [10]:
#Selecting our X and y values from the dataset. Y has the river discharge values
y_orig = ds['dis']
y = y_orig.copy()
X = ds.drop([ 'dis_diff','lsp-4-11','lsp-26-55', 'lsp-12-25','lsp-56-180','cp-1','cp-2','cp-3','sd_diff', 'lsp-1','lsp-2','lsp-3'])


#Xda, yda = X.to_array(dim='features').T, y
Xda, yda = reshape_scalar_predictand(X, y)


In [11]:
X

<xarray.Dataset>
Dimensions:  (time: 13514)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2017-12-31
Data variables:
    cp       (time) float32 0.0027969202 0.0010736011 ... 6.3911815e-05
    lsp      (time) float32 0.0030839704 0.01112505 ... 0.006716862 0.0028783886
    swvl1    (time) float32 0.36975124 0.36971572 ... 0.37917453 0.3986332
    ro       (time) float32 4.0711366e-05 4.0798546e-05 ... 6.1603096e-05
    sd       (time) float32 0.050891068 0.06464155 0.061856583 ... nan nan nan
    dis      (time) float64 798.4 792.2 796.7 814.2 ... 728.3 723.6 720.8 741.5

In [12]:
period_train = dict(time=slice(None, '2005'))
period_valid = dict(time=slice('2006', '2011'))
period_test = dict(time=slice('2012', '2016'))


X_train, y_train = Xda.loc[period_train], yda.loc[period_train]
X_valid, y_valid = Xda.loc[period_valid], yda.loc[period_valid]
X_test, y_test = Xda.loc[period_test], yda.loc[period_test]

In [14]:
import numpy as np

#Applying feature scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train_scaled = sc.fit_transform(X_train)
sc2 = MinMaxScaler(feature_range=(0,1))
y_train_scaled = sc2.fit_transform(y_train.values.reshape(-1,1))

X_train_final, y_train_final = shift_input(X_train_scaled, y_train_scaled, days_intake_length, 1)

In [20]:
y_train_final = y_train_final.reshape(-1)

In [21]:
#Returns NaN values if you don't choose to use reshape_scalar_predictand

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.core import Dropout


regressor = Sequential()

regressor.add(LSTM(units=100, return_sequences= True, input_shape=(X_train_final.shape[1], X_train_final.shape[2])))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units=100, return_sequences= True))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units=100, return_sequences= True))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units=100))
regressor.add(Dropout(0.1))

regressor.add(Dense(units=1))

regressor.compile(optimizer='adam', loss='mean_squared_error')

Using TensorFlow backend.
/opt/miniconda3/envs/flood_prediction/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/miniconda3/envs/flood_prediction/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/miniconda3/envs/flood_prediction/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/minico

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
history = regressor.fit(X_train_final, y_train_final, epochs=50, batch_size=32)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(regressor.predict(X_train))

In [ ]:
plt.plot(y_train_final)

In [ ]:
regressor_yaml = regressor.to_yaml()
with open("../../models/sample-analysis/final-lstm.yaml", "w") as yaml_file:
    yaml_file.write(regressor_yaml)
# serialize weights to HDF5
regressor.save_weights("../../models/sample-analysis/final-lstm.h5")
#Seialize feature scaling weights